In [1]:
import tweepy
import csv 
import os

In [2]:
def read_csv(filename): 
    with open(filename, 'r') as f: 
        reader = csv.DictReader(f)
        for line in reader: 
            yield line

def csv_writer(filename,to_write):
    headers = to_write[0].keys()
    file_exists = os.path.isfile(filename)

    with open(filename,'a') as f:
        writer = csv.DictWriter(f, delimiter=',',fieldnames=headers)
        
        
        if not file_exists:
            writer.writeheader()
            for line in to_write:
                writer.writerow(line)

        
        
def assemble_tweet(tweet_chunk):
    tweets = []

    for i, t in enumerate(tweet_chunk.data): 
        tweet={}
    
        tweet['text'] = t.text
        tweet['author_id'] = t.author_id
        tweet['retweets'] = t.public_metrics['retweet_count']
        tweet['replies'] = t.public_metrics['reply_count']
        tweet['like'] = t.public_metrics['like_count']
        tweet['retweeted_text'] = ''
        tweet['username'] =''
        tweet['created_at'] = t.created_at
        tweet['lang']= t.lang
        tweet['source'] = t.source
    
    
    # trova testo di RT per risolvere il bug del troncamento
    
        if t.referenced_tweets:
            for st in t.referenced_tweets: 
                if st.type =='retweeted':
                    rt_id = st.id
        
        
                    for rt in tweet_chunk.includes['tweets']: 
                        if rt.id == rt_id:
                            tweet['retweeted_text'] = rt.text
    
    
    # trova username autore
   
        for user in tweet_chunk.includes['users']: 
            if user.id == tweet['author_id']: 
                tweet['username']= user.username
    
                    
                    
                    
                    
                    
                    
                    
        tweets.append(tweet)
    
    return tweets
    
    
    

    
    

In [3]:
bearer_token = ''


In [4]:
fields = ['lang','created_at','public_metrics','author_id','referenced_tweets','source']

client = tweepy.Client(bearer_token=bearer_token,wait_on_rate_limit=True)

In [5]:
ids=[]



for row in read_csv('test_ids.csv'):
    ids.append(row['id'])
ids = [ids[i:i + 100] for i in range(0, len(ids), 100)]

In [6]:
tweet_chunks =[]

for i, chunk in enumerate(ids):
    


    result = client.get_tweets(chunk, expansions=['author_id','referenced_tweets.id'], tweet_fields=fields)
    

    tweet_chunks.append(result)
    print('processing chunk n '+str(i))

    

        

Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

In [7]:
tweets=[]


for c in tweet_chunks:
    tweets_t = assemble_tweet(c)
    tweets.extend(tweets_t)

In [33]:
csv_writer('hydrated_tweets.csv',tweets)